In [155]:
staff_no = 8
MONTH = 9
YEAR = 2018

In [156]:
from calendar import Calendar
from collections import Counter
import copy, random
cl = Calendar()

dates = [d for d in cl.itermonthdates(year=YEAR, month=MONTH) if d.month == MONTH]
d = dates[0]
d.isoweekday()

6

In [157]:
staff = {0: 'Γεωργιου',
         1: 'Ανδρονικιδη',
         2: 'Σμυρλη',
         3: 'Πλαβουκου',
         4: 'Παπανικολαου',
         5: 'Καμμενος',
         6: 'Βαβουρακη',
         7: 'Μπεη'
        }

single_shift_people = [2,3,4,7,5]

holidays = {0: [27, 28, 29, 30],
            1: [8, 19, 21, 22, 23, 28, 29, 30],
            2: [1,2,3,4,5,6,7,8,9, 12,13,14,15,28,29],
            3: [1,2,7,8,9,13,14,15],
            4: [1,2,3,4,5,6,7,8,12,13,15,18,20,25,27],
            5: [1,2,13,14,15,16,17],
            6: [1,16,17,18,19,20,21,22,23,27,28,29],
            7: [3,6,7,8,9,10,11,12]
            }

single_shift_dates = {4, 12, 20, 24, 28}

n_shifts = 2*len(dates) - len(single_shift_dates)
shift_names = [(d,suffix) for d in dates for suffix in (['a','b'] if d.day not in single_shift_dates else ['n'])]
shift_day = [d[0].isoweekday() for d in shift_names]
is_weekend_shift = [s in {5,6,7} for s in shift_day]


In [165]:
poss = zip([d[0].day for d in shift_names], [{k for k,v in holidays.items() if (d[0].day not in v) and not((d[1]=='n') and (k not in single_shift_people))} for d in shift_names],[d[1] for d in shift_names] )
poss = [list(p) for p in poss]
# at least 1 fri, sat, sun each
# Can't do two days in a row

In [166]:
poss

[[1, {0, 1, 7}, 'a'],
 [1, {0, 1, 7}, 'b'],
 [2, {0, 1, 6, 7}, 'a'],
 [2, {0, 1, 6, 7}, 'b'],
 [3, {0, 1, 3, 5, 6}, 'a'],
 [3, {0, 1, 3, 5, 6}, 'b'],
 [4, {3, 5, 7}, 'n'],
 [5, {0, 1, 3, 5, 6, 7}, 'a'],
 [5, {0, 1, 3, 5, 6, 7}, 'b'],
 [6, {0, 1, 3, 5, 6}, 'a'],
 [6, {0, 1, 3, 5, 6}, 'b'],
 [7, {0, 1, 5, 6}, 'a'],
 [7, {0, 1, 5, 6}, 'b'],
 [8, {0, 5, 6}, 'a'],
 [8, {0, 5, 6}, 'b'],
 [9, {0, 1, 4, 5, 6}, 'a'],
 [9, {0, 1, 4, 5, 6}, 'b'],
 [10, {0, 1, 2, 3, 4, 5, 6}, 'a'],
 [10, {0, 1, 2, 3, 4, 5, 6}, 'b'],
 [11, {0, 1, 2, 3, 4, 5, 6}, 'a'],
 [11, {0, 1, 2, 3, 4, 5, 6}, 'b'],
 [12, {3, 5}, 'n'],
 [13, {0, 1, 6, 7}, 'a'],
 [13, {0, 1, 6, 7}, 'b'],
 [14, {0, 1, 4, 6, 7}, 'a'],
 [14, {0, 1, 4, 6, 7}, 'b'],
 [15, {0, 1, 6, 7}, 'a'],
 [15, {0, 1, 6, 7}, 'b'],
 [16, {0, 1, 2, 3, 4, 7}, 'a'],
 [16, {0, 1, 2, 3, 4, 7}, 'b'],
 [17, {0, 1, 2, 3, 4, 7}, 'a'],
 [17, {0, 1, 2, 3, 4, 7}, 'b'],
 [18, {0, 1, 2, 3, 5, 7}, 'a'],
 [18, {0, 1, 2, 3, 5, 7}, 'b'],
 [19, {0, 2, 3, 4, 5, 7}, 'a'],
 [19, {0, 2, 3

In [217]:
def generate_random_solution(poss_in, counter_in):
    if len(poss_in) == 0:
        raise ValueError
        
    if not poss_in[0][1]:
        yield None
    
    choices = list(poss_in[0][1]) 
    
    random.shuffle(choices)
    
    if len(poss_in) == 1:
        for i in choices:
            yield [i]
    else:
        for item in choices:
            poss = copy.deepcopy(poss_in)
            counter = copy.deepcopy(counter_in)
            shift_day = poss[0][0]
            shift_suffix = poss[0][2]
            shift_to_remove = 0
            counter[item] += 1
            while True:
                shift_to_remove += 1
                if shift_to_remove >= len(poss):
                    break
                next_shift_day = poss[shift_to_remove][0]
                if next_shift_day > shift_day + 1:
                    break
                else:
                    poss[shift_to_remove][1] -= {item}
                
            if shift_suffix == 'n':
                shift_to_remove = 0
                for i,p in enumerate(poss):
                    if p[2] == 'n':
                        poss[i][1] -= {item}
                        
            if counter[item] == 7:
                for i,p in enumerate(poss):
                    poss[i][1] -={item}

            soln = generate_random_solution(poss[1:], counter)
            for s in soln:
                if s:
                    yield [item] + s
    

In [218]:
def calculate_weekend_allowance(poss):
    return {'Fri': Counter([p for p,s in zip(poss, shift_day) if s == 5]),
            'Sat': Counter([p for p,s in zip(poss,shift_day) if s == 6]),
            'Sun': Counter([p for p,s in zip(poss, shift_day) if s == 7]),
            'Total': Counter([p for p,s in zip(poss, is_weekend_shift) if s])}

def is_valid(poss):
    wkds = calculate_weekend_allowance(poss)
    people = staff.keys()
    r = 0
    for k in ['Fri', 'Sat', 'Sun']:
        for p in people:
            if wkds[k][p] == 0:
                r += 1
    
    single_shift_people = Counter(p for p,s in zip(poss, shift_names) if shift_names[1] == 'n')
    for k,v in single_shift_people.items():
        if v > 1:
            r += 20*(v-1)**2
    
    return r

In [221]:
c = Counter({k:0 for k in staff.keys()})
gen = generate_random_solution(poss,c)

In [222]:
min_score = 100
best_solns = []

while True:
    sol = next(gen)
    score = is_valid(sol)
    if score < min_score:
        best_solns = [sol]
        min_score = score
        print("Score: {}".format(score))
    if score == min_score:
        best_solns.append(sol)
        
    if score == 0:
        break




Score: 7
Score: 5
Score: 4
Score: 3


KeyboardInterrupt: 

In [205]:
from itertools import groupby
from collections import OrderedDict
def print_solution(sol):
    for day, people_group in groupby(zip(shift_names,sol), key=lambda x: x[0][0]):
        print('{day}: {name}'.format(day=str(day), name=', '.join(list(staff[p[1]] for p in people_group))))
    
    wkd = calculate_weekend_allowance(sol)
    
    p_dict = {staff[k]: OrderedDict([('Παρ', wkd['Fri'][k]), ('Σαβ',wkd['Sat'][k]), ('Κυρ',wkd['Sun'][k])]) for k in staff.keys()}
    display(pd.DataFrame(p_dict))
    
    totals = Counter(sol)
    totals = {staff[k]:{'count':v} for k,v in totals.items()}
    display(pd.DataFrame(totals))

In [223]:
len(best_solns)

3649

In [224]:
from IPython.display import display
import pandas as pd

In [228]:
def max_wkd_days(soln):
    wkd = calculate_weekend_allowance(sol)
    return max(max(wkd['Fri'].values()), max(wkd['Sat'].values()), max(wkd['Sun'].values()))
    
sol = min(best_solns, key=lambda x: max_wkd_days(x))


In [229]:
print_solution(sol)

2018-09-01: Γεωργιου, Μπεη
2018-09-02: Ανδρονικιδη, Βαβουρακη
2018-09-03: Καμμενος, Γεωργιου
2018-09-04: Μπεη
2018-09-05: Καμμενος, Βαβουρακη
2018-09-06: Γεωργιου, Πλαβουκου
2018-09-07: Καμμενος, Ανδρονικιδη
2018-09-08: Γεωργιου, Βαβουρακη
2018-09-09: Παπανικολαου, Ανδρονικιδη
2018-09-10: Πλαβουκου, Βαβουρακη
2018-09-11: Γεωργιου, Σμυρλη
2018-09-12: Καμμενος
2018-09-13: Ανδρονικιδη, Μπεη
2018-09-14: Παπανικολαου, Βαβουρακη
2018-09-15: Ανδρονικιδη, Γεωργιου
2018-09-16: Μπεη, Σμυρλη
2018-09-17: Γεωργιου, Παπανικολαου
2018-09-18: Μπεη, Σμυρλη
2018-09-19: Πλαβουκου, Καμμενος
2018-09-20: Σμυρλη
2018-09-21: Παπανικολαου, Μπεη
2018-09-22: Σμυρλη, Πλαβουκου
2018-09-23: Μπεη, Καμμενος
2018-09-24: Παπανικολαου
2018-09-25: Σμυρλη, Βαβουρακη
2018-09-26: Παπανικολαου, Πλαβουκου
2018-09-27: Σμυρλη, Ανδρονικιδη
2018-09-28: Πλαβουκου
2018-09-29: Παπανικολαου, Καμμενος
2018-09-30: Βαβουρακη, Πλαβουκου


,Γεωργιου,Ανδρονικιδη,Σμυρλη,Πλαβουκου,Παπανικολαου,Καμμενος,Βαβουρακη,Μπεη
Κυρ,0,2,1,1,1,1,2,2
Παρ,0,1,0,1,2,1,1,1
Σαβ,3,1,1,1,1,1,1,1


,Γεωργιου,Μπεη,Ανδρονικιδη,Βαβουρακη,Καμμενος,Πλαβουκου,Παπανικολαου,Σμυρλη
count,7,7,6,7,7,7,7,7
